In [ ]:
!pip install openai pandas
!pip install xlrd openpyxl
!pip install statsmodels

In [20]:
from openai import OpenAI
import pandas as pd
import statsmodels as st

In [ ]:
def send_prompt(prompt):
    client = OpenAI(api_key="<key>", base_url="https://api.deepseek.com")
    try:
        response = client.chat.completions.create(
            model="deepseek-reasoner",
            messages=[
                {"role": "system", "content": "You are a scientist"},
                {"role": "user", "content": prompt},
            ],
            stream=False
        )
        content = response.choices[0].message.content
        return content
    except Exception as e:
        content = "Não foi possível realizar a consulta."
        return content

In [ ]:
with open("classificacao_deepseek_r1_05_04.csv", 'w') as file:
    file.write("title|abstract|reason|answer\n")

for titulo, resumo in deep_seek_selecao.values:
    resumo_limpo = resumo.replace("\n", " ").rstrip().strip()
    titulo_limpo = titulo.replace("{", "").replace("}", "")
    prompt = f"""Given the title of a scientific article and its abstract, determine whether the article should be included or excluded from the dataset.
    To decide, consider that for an article to be included, it must:
    - Use Concept Maps or Knowledge Maps as a teaching and learning tool in any educational environment.
    - Perform any type of automated assessment of Concept Maps.
    
    Don't include an article if it just assess or create concept maps automatically, it must assess concept maps created by students.
    
    To respond, follow this exact format:
    Provide an explanation of why the article should be included or excluded.
    Ensure the response is formatted exactly as follows:
    answer=included if the article should be included;
    answer=excluded if it should be excluded;

    These are the article details you must analyze:
    Title: {titulo_limpo}
    Abstract: {resumo_limpo}' 
    """
    content = send_prompt(prompt)
    
    if "answer=" not in content:
        print("falhou com artigo", titulo_limpo)
        string_resultado = f"{titulo_limpo}|{resumo_limpo}|{content.strip()}|error\n"
        with open("classificacao_deepseek_r1_05_04.csv", "a") as file:
            file.write(string_resultado)
    else:
        texto = content.split("answer=")
        motivo = texto[0].replace("\n", "").strip()
        resposta = texto[1].replace("\n", "").strip()
        string_resultado = f"{titulo_limpo}|{resumo_limpo}|{motivo}|{resposta}\n"
        with open("classificacao_deepseek_r1_05_04.csv", "a") as file:
            file.write(string_resultado)

---

## Fleiss' Kappa

In [4]:
selecao_p1 = pd.read_excel("review_p1.xls")
selecao_p1 = selecao_p1.loc[selecao_p1.status != "Duplicated"]
selecao_p1.title = selecao_p1.title.apply(lambda x: x.replace("{", "").replace("}", "").lower().replace("’", "'"))
selecao_p2 = pd.read_csv("selecao_p2.csv")
selecao_p2.title = selecao_p2.title.apply(lambda x: x.replace("{", "").replace("}", "").lower().replace("’", "'"))
selecao_deepseek = pd.read_csv('classificacao_deepseek_r1_05_04.csv', sep="|")

In [8]:
lista_titulos_select = []
for item in selecao_deepseek.title.values.tolist():
    lista_titulos_select.append(item.lower().replace("’", "'"))

In [9]:
selecao_p1_cortada = selecao_p1.loc[selecao_p1.title.isin(lista_titulos_select)]

In [11]:
selecao_p1_cortada = selecao_p1_cortada.sort_values(by='title')
selecao_p2 = selecao_p2.sort_values(by='title')
selecao_deepseek = selecao_deepseek.sort_values(by='title')

In [12]:
selecao_p1_cortada['resposta_numerica'] = selecao_p1_cortada['status'].apply(lambda x: 0 if x == 'Rejected' else 1)
selecao_p2['resposta_numerica'] = selecao_p2['Avaliação'].apply(lambda x: 0 if x == 'Rejected' else 1)
selecao_deepseek['resposta_numerica'] = selecao_deepseek['answer'].apply(lambda x: 0 if x == 'excluded' else 1)

In [ ]:
import numpy as np
from statsmodels.stats import inter_rater as irr

avaliador1 = selecao_p1_cortada['resposta_numerica'].values.tolist()
avaliador2 = selecao_p2['resposta_numerica'].values.tolist()
avaliador3 = selecao_deepseek['resposta_numerica'].values.tolist()
# Número de artigos e avaliadores
n_artigos = len(avaliador1)
n_avaliadores = 3

# Inicializar a matriz de contagem
matriz_contagem = np.zeros((n_artigos, 2))  # 2 categorias: 0 e 1

# Preencher a matriz de contagem
for i in range(n_artigos):
    # Contar quantos avaliadores marcaram 0 (rejeitado) e 1 (aceito) para o artigo i
    count_0 = (avaliador1[i] == 0) + (avaliador2[i] == 0) + (avaliador3[i] == 0)
    count_1 = (avaliador1[i] == 1) + (avaliador2[i] == 1) + (avaliador3[i] == 1)
    
    matriz_contagem[i, 0] = count_0
    matriz_contagem[i, 1] = count_1

# Calcular o Fleiss' Kappa
kappa = irr.fleiss_kappa(matriz_contagem)

print("Matriz de contagem:")
print(matriz_contagem)
print("\nFleiss' Kappa:", kappa)